# pandas operations

In this notebook:

- Data aggregation: `groupby()`
- Joining `DataFrame` objects: `merge()`
- Basics of data visualisation with pandas: `plot()`

In [ ]:
import pandas as pd

clients_file = '../data/fake_shop/fake_clients.csv'
transactions_file = '../data/fake_shop/fake_transactions.csv'

clients = pd.read_csv(clients_file, parse_dates=['date_of_birth'])
transactions = pd.read_csv(transactions_file, parse_dates=['date'])

## Data aggregation

This section is about data aggregation, using the `groupby()` function.

If you're familiar with SQL, there are similarities with `GROUP BY` queries.

First, let's look at the structure of the transaction `DataFrame`.

In [ ]:
transactions.head()

Given this schema, we might be interested in aggregating data per client.

The `groupby()` function creates an intermediate object that we can use to run downstream aggregation functions.

In [ ]:
transactions.groupby('client_id')

In [ ]:
transactions.groupby('client_id').sum()

Depending on the specific aggregation function, `sum()` in the example above, we may want to focus on specific columns where the function makes more sense, e.g. summing up transaction_id is not useful.

We can do a projection on the column(s) of interest before running the aggregation.

**Example**: compute the grand-total for each customer

In [ ]:
transactions.groupby('client_id')['total'].sum()

**Example**: compute the total cost of each transaction.

In [ ]:
transactions.groupby('transaction_id')['total'].sum()

**Example**: compute the average total cost per transaction:

In [ ]:
transactions.groupby('transaction_id')['total'].sum().mean()

**Example**: compute the number of different shopping days per customer

In [ ]:
transactions.groupby('client_id')['date'].nunique()

The `groupby()` function also allows to group by multiple fields.

**Example**: compute the amount spent by each customer on different products:

In [ ]:
transactions.groupby(['client_id', 'product'])['total'].sum()

#### List of aggregation functions

- `sum()` for the arithmetic sum
- `mean()` for the arithmetic average
- `count()` for the count of records (`NaN` not included)
- `size()` for the count of records including `NaN`
- `nunique()` for the number of unique records
- `var()` and `std()` for variance and standard deviation
- `min()` and `max()` for minimum and maximum
- `first()`, `last()`, `nth()` for the first, last and Nth record
- `describe()` for overall descriptive statistics

### Applying multiple aggregation function at once

The function `aggregate()` (or simply `agg()` in short) allows to pass multiple functions at once.

**Example**: compute the total cost and number of different products for each transaction

In [ ]:
import numpy as np

transactions.groupby('transaction_id')['total'].agg([np.sum, np.size])

We can also explicitly name the column labels for the output:

In [ ]:
transactions.groupby('transaction_id')['total'].agg(total_cost=np.sum,
                                                    n_different_products=np.size)  # label_name=function_name

#### References:

https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html

## Joining `DataFrame` objects

This section is about joining `DataFrame` objects.

If you're familiar with SQL, you'll find similarities with `JOIN` queries.

A joining operation is used to combine records from different `DataFrame` objects, based on either the index or the value of some columns.

Firstly, let's consider the structure of the tables: both have a `client_id` column, and this will be our joining key.

The joining operation can be performed with the function `merge()`.

**Note**: like the SQL equivalent, `merge()` uses a notion of left/right tables, which affects the order of the columns in output. 

In [ ]:
pd.merge(left=clients, right=transactions, on='client_id')

For cases where the joining key has different names in the different tables, we can use `left_on` and `right_on`:

In [ ]:
pd.merge(left=clients, right=transactions, left_on='client_id', right_on='client_id')

There are different types of joins:

- `inner` (default): only records with matching values on both sides will be included
- `left`: all records from the left table are included, if there's no match on the right table then fill the blanks with `NaN`
- `right`: all records from the right table are included, if there's no match on the left table then fill the blanks with `NaN`
- `outer`: all records from both tables are included, if there's no match on the other table then fill the blanks with `NaN`

This behaviour is modified using the `how` argument, e.g.

In [ ]:
pd.merge(left=clients, right=transactions, left_on='client_id', right_on='client_id', how='outer')

## Basics of data visualisation with pandas

pandas offers a nice integration with data visualisation libraries (matplotlib by default).

pandas `DataFrame` objects support a method `plot()` which allows you to easily visualise your data.

#### References:

https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html

https://pandas.pydata.org/pandas-docs/stable/user_guide/cookbook.html#cookbook-plotting

https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.plot.html

In [ ]:
# The following line tells matplotlib to visualise the output within the notebook
%matplotlib inline

In [ ]:
# Default: line chart
# This calls matplotlib.pyplot.plot()
transactions['product'].value_counts().plot()

Note: to hide the reference to the matplotlib object, use a `;` at the end of the line

In [ ]:
# This calls matplotlib.pyplot.bar()
transactions['product'].value_counts().plot(kind='bar');

In [ ]:
# This calls matplotlib.pyplot.pie()
transactions['product'].value_counts().plot(kind='pie');

In [ ]:
# blue, red, green, yellow, black repeated twice
custom_colours = ['b', 'r', 'g', 'y', 'k'] * 2

clients['city'].value_counts().plot(kind='bar', color=custom_colours);

The following codes are available from the matplotlib library:
- `b` for blue
- `r` for red
- `g` for green
- `y` for yellow
- `k` for black
- `c` for cyan
- `m` for magenta
- `w` for white

It's also possible to pass colours encoded as RGB or RGB-Alpha using tuples.

The values must be normalised in a \[0-1\] range.

In [ ]:
def normalise(colour):
    """Take a colour tuple in the form (R, G, B) and normalise it.
    
    The colours are given in a 0-255 range and returned in a 0-1 range.
    """
    return (colour[0] / 255, colour[1] / 255, colour[2] / 255)
    
black = (0, 0, 0)
dark_grey = (87, 87, 87)
red = (173, 35, 35)
blue = (42, 75, 215)
green = (29, 105, 20)
brown = (129, 74, 25)
purple = (129, 38, 192)
light_grey = (160, 160, 160)
light_green = (129, 197, 122)
light_blue = (157, 175, 255)
cyan = (41, 208, 208)
orange = (255, 146, 51)
yellow = (255, 238, 51)
beige = (233, 222, 187)
pink = (255, 205, 243)
white = (255, 255, 255)

# 15 colours, not including white
colormap = [black, yellow, blue, green, brown, red, light_grey, orange, light_green, beige, light_blue, purple, pink, cyan, dark_grey]

colormap = [normalise(c) for c in colormap]

clients['city'].value_counts().plot(kind='bar', color=colormap);


In [ ]:
transactions['product'].value_counts().plot(kind='pie', colors=colormap);

#### Other customisations

The function `plot()` takes also other arguments to customise the plot.

The output of the `plot()` function is the matplotlib graph itself, so further customisations are possible using the matplotlib API.

In [ ]:
ax = clients['city'].value_counts().plot(kind='bar',
                                         color=colormap,
                                         figsize=(12, 6),
                                         title='Number of clients per city')
ax.set_xlabel('Cities')
ax.set_ylabel('# clients');

## Exercises

Once you are familiar with the concepts described in this notebook, please continue with the following notebook:

[Exercises on pandas operations](02.1%20-%20Exercises%20on%20pandas%20operations.ipynb)